In [ ]:
import emat
emat.versions()

# Feature Scoring

Feature scoring is a methodology for identifying what model inputs (in machine 
learning terminology, “features”) have the greatest relationship to the outputs.  
The relationship is not necessarily linear, but rather can be any arbitrary 
linear or non-linear relationship.  For example, consider the function:

In [ ]:
import numpy

def demo(A=0,B=0,C=0,**unused):
    """
    Y = A/2 + sin(6πB) + ε
    """
    return {'Y':A/2 + numpy.sin(6 * numpy.pi * B) + 0.1 * numpy.random.random()}

We can readily tell from the functional form that the *B* term is the
most significant when all parameter vary in the unit interval, as the 
amplitude of the sine wave attached to *B* is 1 (although the relationship 
is clearly non-linear) while the maximum change
in the linear component attached to *A* is only one half, and the output
is totally unresponsive to *C*.

To demonstrate the feature scoring, we can define a scope to explore this 
demo model:

In [ ]:
demo_scope = emat.Scope(scope_file='', scope_def="""---
scope:
    name: demo
inputs:
    A:
        ptype: exogenous uncertainty
        dtype: float
        min: 0
        max: 1
    B:
        ptype: exogenous uncertainty
        dtype: float
        min: 0
        max: 1
    C:
        ptype: exogenous uncertainty
        dtype: float
        min: 0
        max: 1
outputs:
    Y:  
        kind: info
""")

And then we will design and run some experiments to generate data used for
feature scoring.

In [ ]:
from emat import PythonCoreModel
demo_model = PythonCoreModel(demo, scope=demo_scope)
experiments = demo_model.design_experiments(n_samples=5000)
experiment_results = demo_model.run_experiments(experiments)

In [ ]:
from emat.analysis import feature_scores
fs = feature_scores(demo_scope, experiment_results)
fs

In [ ]:
from emat.analysis import display_experiments
fig = display_experiments(demo_scope, experiment_results, render=False, return_figures=True)['Y']
fig.update_layout(
    xaxis_title_text =f"A (Feature Score = {fs.data.loc['Y','A']:.3f})",
    xaxis2_title_text=f"B (Feature Score = {fs.data.loc['Y','B']:.3f})",
    xaxis3_title_text=f"C (Feature Score = {fs.data.loc['Y','C']:.3f})",
)
from emat.util.rendering import render_plotly
render_plotly(fig, '.png')

One important thing to consider is that changing the range of the input parameters 
in the scope can significantly impact the feature scores, even if the underlying 
model itself is not changed.  For example, consider what happens to the features
scores when we expand the range of the uncertainties:

In [ ]:
demo_model.scope = emat.Scope(scope_file='', scope_def="""---
scope:
    name: demo
inputs:
    A:
        ptype: exogenous uncertainty
        dtype: float
        min: 0
        max: 5
    B:
        ptype: exogenous uncertainty
        dtype: float
        min: 0
        max: 5
    C:
        ptype: exogenous uncertainty
        dtype: float
        min: 0
        max: 5
outputs:
    Y:  
        kind: info
""")

In [ ]:
wider_experiments = demo_model.design_experiments(n_samples=5000)
wider_results = demo_model.run_experiments(wider_experiments)

In [ ]:
from emat.analysis import feature_scores
wider_fs = feature_scores(demo_model.scope, wider_results)
wider_fs

In [ ]:
fig = display_experiments(demo_model.scope, wider_results, render=False, return_figures=True)['Y']
fig.update_layout(
    xaxis_title_text =f"A (Feature Score = {wider_fs.data.loc['Y','A']:.3f})",
    xaxis2_title_text=f"B (Feature Score = {wider_fs.data.loc['Y','B']:.3f})",
    xaxis3_title_text=f"C (Feature Score = {wider_fs.data.loc['Y','C']:.3f})",
)
render_plotly(fig, '.png')

The pattern has shifted, with the sine wave in *B* looking much more like the random noise,
while the linear trend in *A* is now much more important in predicting the output, and
the feature scores also shift to reflect this change.

## Road Test Feature Scores

We can apply the feature scoring methodology to the Road Test example 
in a similar fashion.  To demonstrate scoring, we'll first load and run
a sample set of experients.

In [ ]:
from emat.model.core_python import Road_Capacity_Investment
road_scope = emat.Scope(emat.package_file('model','tests','road_test.yaml'))
road_test = PythonCoreModel(Road_Capacity_Investment, scope=road_scope)
road_test_design = road_test.design_experiments(sampler='lhs')
road_test_results = road_test.run_experiments(design=road_test_design)

In [ ]:
feature_scores(road_scope, road_test_results)

In [ ]:
feature_scores(road_scope, road_test_results, return_type='dataframe')

In [ ]:
feature_scores(road_scope, road_test_results, return_type='figure')

The colors on the returned DataFrame highlight the most important input features
for each performance measure (i.e., in each row).  The yellow highlighted cell 
indicates the most important input feature for each output feature, and the 
other cells are colored from yellow through green to blue, showing high-to-low
importance.  These colors are from matplotlib's default "viridis" colormap. 
A different colormap can be used by giving a named colormap in the `cmap`
argument.

In [ ]:
feature_scores(road_scope, road_test_results, cmap='copper')

You may also notice small changes in the numbers given in the two tables above. This
occurs because the underlying algorithm for scoring uses a random trees algorithm. If
you need to have stable (replicable) results, you can provide an integer in the 
`random_state` argument.

In [ ]:
feature_scores(road_scope, road_test_results, random_state=1, cmap='bone')

Then if we call the function again with the same `random_state`, we get the same numerical result.

In [ ]:
feature_scores(road_scope, road_test_results, random_state=1, cmap='YlOrRd')

## Interpreting Feature Scores

The correct interpretation of feature scores is obviously important.  As noted above,
the feature scores can reveal both linear and non-linear relationships. But the scores
themselves give no information about which is which. 

In addition, while the default feature scoring algorithm generates scores that total 
to 1.0, it does not necessarily map to dividing up the explained variance. Factors that
have little to no effect on the output still are given non-zero feature score values.
You can see an example of this in the "demo" function above; that simple example 
literally ignores the "C" input, but it has a non-zero score assigned.  If there are
a large number of superfluous inputs, they will appear to reduce the scores attached
to the meaningful inputs.

It is also important to remember that these scores do not fully reflect 
any asymmetric relationships in the data. A feature may be very important for some portion
of the range of a performance measure, and less important in other parts of the range.
For example, in the Road Test model, the "expand_capacity" lever has a highly asymmetric
impact on the "net_benefits" measure: it is very important in
determining negative values (when congestion isn't going to be bad due to low "input_flow" volumes, 
the net loss is limited by how much we spend), but not so important for positive values 
(nearly any amount of expansion has a big payoff if congestion is going to be bad due 
to high "input_flow" volume).  If we plot this three-way relationship specifically, we can 
observe it on one figure:

In [ ]:
from matplotlib import pyplot as plt
fig, ax = plt.subplots()
ax = road_test_results.plot.scatter(
    c='net_benefits', 
    y='expand_capacity', 
    x='input_flow', 
    cmap='coolwarm',
    ax=ax,
)

Looking at the figure above, we can see the darker red clustered to the right,
and the darker blue clustered in the top left.
However, if we are not aware of this particular three-way relationship
*a priori*, it may be difficult to discover it by looking through various
combinations of three-way relationships.  To uncover this kind of relationship,
threshold scoring may be useful.

## Threshold Scoring

In [ ]:
from emat.analysis.feature_scoring import threshold_feature_scores

threshold_feature_scores(road_scope, 'net_benefits', road_test_results)

In [ ]:
threshold_feature_scores(road_scope, 'net_benefits', road_test_results, return_type='figure.png')

In [ ]:
threshold_feature_scores(road_scope, 'net_benefits', road_test_results, return_type='ridge figure.svg')

In these figures, we can see that "expand_capacity" is important for negative outcomes,
but for positive outcomes we should focus more on "input_flow", and to a lesser but
still meaningful extent also "value_of_time".

## Feature Scoring API 